In [97]:
import os
import h5py
import numpy as np


In [98]:
PATH = '../EOS/compOSE/FOP(SFHoY)/'
# files = [file for file in os.listdir(PATH) if 'eos.' in file and not '.pdf' in file and not '.init' in file]
# files

files = ['eos.nb', 'eos.thermo', 'eos.t', 'eos.yq']

In [99]:
skip_rows = {
                'eos.nb' : 2,
                'eos.thermo' : 1,
                'eos.t' : 2,
                'eos.yq' : 2
            }

data = {}
for file in files:
    data[file] = np.loadtxt(PATH + file, skiprows=skip_rows[file])
    if file == 'eos.thermo':
        with open(PATH + file) as f:
            m_n, m_p, _ = np.fromstring(f.readline().strip('\n'), dtype = float, sep='\t')

In [100]:
# Gli indici sono da 1 a N in Fortran, da 0 a N-1 in Python
index_T    = data['eos.thermo'][:, 0].astype(int) - 1
index_nb   = data['eos.thermo'][:, 1].astype(int) - 1
index_ye   = data['eos.thermo'][:, 2].astype(int) - 1

rho        = data['eos.nb'] * m_n     # LOGARITMICO     MeV / fm^3
pointsrho  = len(rho)

y_e        = data['eos.yq']           # LINEARE         Adimensionale
pointsye   = len(y_e)

temp       = data['eos.t']            # LOGARITMICO     MeV
pointstemp = len(temp)

maps = lambda i,j,k : i + pointsrho * pointsye * j + pointsye * k


## COMPUTE PRESSURE, RESHAPE AND TEST FOR EQUIVALENCE!!! Secondo metodo circa 28 volte più veloce del primo

pressure   = data['eos.thermo'][:, 3] * data['eos.nb'][index_nb]   # MeV / fm^3
pp = np.zeros((pointsye, pointstemp, pointsrho))
for i in range(pointsye):
    for j in range(pointstemp):
        for k in range(pointsrho):
            pp[i, j, k] = pressure[maps(i,j,k)]

pressure_r = pressure[maps(np.arange(pointsye)[:, None, None], np.arange(pointstemp)[None, :, None], np.arange(pointsrho)[None, None, :]) ]

np.testing.assert_allclose(pressure_r, pp)

#############################################
## COMPUTE ENTROPY, RESHAPE AND TEST FOR EQUIVALENCE!!! Secondo metodo circa 28 volte più veloce del primo

entropy    = data['eos.thermo'][:, 4] * data['eos.nb'][index_nb]   # Adimensionale
ee = np.zeros((pointsye, pointstemp, pointsrho))
for i in range(pointsye):
    for j in range(pointstemp):
        for k in range(pointsrho):
            ee[i, j, k] = entropy[maps(i,j,k)]

entropy_r = entropy[maps(np.arange(pointsye)[:, None, None], np.arange(pointstemp)[None, :, None], np.arange(pointsrho)[None, None, :]) ]

np.testing.assert_allclose(entropy_r, ee)

#############################################
## COMPUTE ENERGY, RESHAPE AND TEST FOR EQUIVALENCE!!! Secondo metodo circa 28 volte più veloce del primo
# Se ho capito bene Q7 --> data['eos.thermo'][:, 9] è esattamente "energy" negli h5 (da capire come la scala)
energy     = data['eos.thermo'][:, 9]
en = np.zeros((pointsye, pointstemp, pointsrho))
for i in range(pointsye):
    for j in range(pointstemp):
        for k in range(pointsrho):
            en[i, j, k] = energy[maps(i,j,k)]

energy_r = energy[maps(np.arange(pointsye)[:, None, None], np.arange(pointstemp)[None, :, None], np.arange(pointsrho)[None, None, :]) ]

np.testing.assert_allclose(energy_r, en)


In [104]:
ds = {'rho':rho, 'pointsrho':pointsrho, 'y_e':y_e, 'pointsye':pointsye, 'temp':temp, 'pointstemp':pointstemp, 'pressure':pressure_r, 'entropy':entropy_r, 'energy':energy_r}

In [ ]:
with h5py.File(PATH + "TEST_FOP(SFHoY).h5", "w") as f:
    for d in ds:
        dset = f.create_dataset(d, data=ds[d], dtype='f')